In [9]:
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization 
from keras.regularizers import WeightRegularizer, ActivityRegularizer 
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
import h5py

In [6]:
import os
directory="./tiny-imagenet-200/train/"
all_folder=os.listdir(directory)
all_folder_real=[x for x in all_folder if x[0]=='n']

for sr in all_folder_real:
    raw="python image_resizer_imagent.py -i ./train/"+sr+"  -o ./data -s 32 -a box -r -j 10"
    print(raw)

python image_resizer_imagent.py -i ./train/n01443537  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01629819  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01641577  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01644900  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01698640  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01742172  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01768244  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01770393  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01774384  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01774750  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01784675  -o ./data -s 32 -a box -r -j 10
python image_resizer_imagent.py -i ./train/n01855672  

In [7]:
loss_functions = ['categorical_crossentropy','squared_hinge','hinge']
num_classes = 200
batch_size = 32
nb_epoch = 30


In [13]:
#Load images
path='../tiny-imagenet-200/train/'
X_train,y_train,X_test,y_test=load_img(path,num_classes)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

FileNotFoundError: [Errno 2] No such file or directory: '../tiny-imagenet-200/train/'

In [14]:
num_samples=len(X_train)
num_channels , img_rows, img_cols = X_train.shape[1], X_train.shape[2], X_train.shape[3]


NameError: name 'X_train' is not defined

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


In [ ]:
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
for loss_function in loss_functions:
    model = Sequential()
    model.add(Convolution2D(512,(5,5),padding='same', input_shape=(32,32,3),subsample=(2,2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    model.add(Activation('relu')) 

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512,(3,3),subsample=(2,2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    model.add(Activation('relu')) 

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256,(3,3),subsample=(2,2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    model.add(Activation('relu')) 
    model.add(Dropout(0.25)) 

    model.add(Convolution2D(256,(3,3),subsample=(1,1)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    model.add(Activation('relu')) 
    model.add(Flatten())
    model.add(Dense(512,W_regularizer=WeightRegularizer(l1=1e-5,l2=1e-5)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    model.add(Activation('softmax')) 
    model.add(Dropout(0.5)) 

In [ ]:
if loss_function=='categorical_crossentropy':
        model.add(Dense(num_classes,activation='softmax',W_regularizer=WeightRegularizer(l1=1e-5,l2=1e-5)))
        sgd = SGD(lr=0.05, decay=1e-5, momentum=0.95, nesterov=True)
    else:
        if loss_function=='hinge':
            sgd = SGD(lr=0.01, decay=1e-6, momentum=0.95, nesterov=True)
        else:
            sgd = SGD(lr=0.1, decay=1e-6, momentum=0.95, nesterov=True)
            model.add(Dense(num_classes,W_regularizer=WeightRegularizer(l1=1e-5,l2=1e-5)))

In [ ]:
model.compile(loss=loss_function,
                  optimizer=sgd,
                  metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
              validation_data=(X_test, Y_test))

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
history= model.fit(x_train, y_train,
batch_size=batch_size,
epochs=10,
validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
history_dict=history.history
history_dict.keys()
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()